### Brazil Forest Fires
#### By Tshepo Rampai

### Context

* Wildfire is a fire in an area of burning vegetation that occurs in the forest or countryside
* Brazil is prone to large forest fires, with it having the largest rainforest in the world

### Content
* The dataset comprises of forest fires per state over a period of over 15 years



### Required

* Format months to English

* Determine which state has the most fires

* Determine which year has the most fires

* Determine which year and month has the most fires


### Required Packages
* plotly == 4.5.3

* pandas == 1.0.1

* numpy == 1.16.2

* dash == 1.10.0

* bokeh == 1.0.4



In [1]:
import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from dash.dependencies import Input, Output
app =  dash.Dash()
server =  app.server
colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

### 1. Cleaning and renaming months

In [2]:
df = pd.read_csv('data/amazon.csv',encoding='latin1')

#MONTHS TO ENGLISH
span_to_eng = {
    'Janeiro':'January',
    'Fevereiro': 'February',
    'Março':'March',
    'Abril':'April',
    'Maio':'May',
    'Junho':'June',
    'Julho':'July',
    'Agosto':'August',
    'Setembro':'September',
    'Outubro': 'October',
    'Novembro': 'November',
    'Dezembro':'December'    
}
df['month'] = df['month'].map(span_to_eng)
df.isnull().sum()

year      0
state     0
month     0
number    0
date      0
dtype: int64

### 2. State that had the most fires

In [3]:
#grouped dataframe
df_max = pd.DataFrame(df.groupby(['year','state','month']).number.sum())

#total by state
df_sum_state = pd.DataFrame(df_max.groupby(['state']).number.sum())
df_sum_state = df_sum_state.sort_values(by=['number'], ascending=False)
df_sum_state = df_sum_state.reset_index(drop=False)
df_sum_state.head()

,state,number
0,Mato Grosso,96246.028
1,Paraiba,52435.918
2,Sao Paulo,51121.198
3,Rio,45160.865
4,Bahia,44746.226


### 3. Year that had the most fires

In [4]:
#total by year
df_sum_year = pd.DataFrame(df_max.groupby(['year']).number.sum())
df_sum_year = df_sum_year.sort_values(by=['number'],ascending=False)
df_sum_year = df_sum_year.reset_index(drop=False)
df_sum_year.head()

,year,number
0,2003,42760.674
1,2016,42212.229
2,2015,41208.292
3,2012,40084.860
4,2014,39621.183


### 4. Year and month with the most fires

In [5]:
#total by month and year
df['key'] = df.month.map(str)+'-'+df.year.apply(str)
df_sum_yr_month = pd.DataFrame(df.groupby(['key']).number.sum())
df_sum_yr_month = df_sum_yr_month.sort_values(by=['number'], ascending=False)
df_sum_yr_month = df_sum_yr_month .reset_index(drop=False)
df_sum_year.head()

,year,number
0,2003,42760.674
1,2016,42212.229
2,2015,41208.292
3,2012,40084.860
4,2014,39621.183


### 5. Dashboard Visualisation

In [6]:
style_table = {
    'maxHeight': '50ex',
    'overflowY': 'scroll',
    'width': '30%',
    'minWidth': '30%',
}
style_cell = {
    'fontFamily': 'Open Sans',
    'textAlign': 'center',
    'height': '30px',
    'padding': '10px 22px',
    'whiteSpace': 'inherit',
    'overflow': 'hidden',
                'textOverflow': 'ellipsis',
}
style_cell_conditional = [
    {
        'if': {'column_id': 'State'},
        'textAlign': 'left'
    },
]
style_header = {
    'fontWeight': 'bold',
    'backgroundColor': "#3D9970",
    'fontSize': '16px'
}
style_data_conditional = [
    {
        'if': {'row_index': 'odd'},
        'backgroundColor': 'rgb(248, 248, 248)'
    }]
style_table = {
    'maxHeight': '50ex',
    'overflowY': 'scroll',
    'width': '100%',
    'minWidth': '100%',
}

In [7]:

# Style sheet
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# App
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# layout
app.layout = html.Div(children=[
    html.H1(
        children='Brazil Forest Fires',
        style={
            'textAlign': 'center',
            'color': 'green'
        }
    ),
    dcc.Tabs(id='tabs-example', value='tab', children=[
        dcc.Tab(label='Fire per state', value='tab-1'),
        dcc.Tab(label='Fires per year', value='tab-2'),
        dcc.Tab(label='Fires per year and month', value='tab-3'),
    ]),
    html.Div(id='tabs-example-content')
])

In [ ]:
# callback for the responsiveness of the tab

@app.callback(Output('tabs-example-content', 'children'),
              [Input('tabs-example', 'value')])

def render_content(tab):
    '''Display the table and the graph sideby
    side when the respective tab is selected'''

    if tab == 'tab-1':
        return html.Div([
            html.Div([
                html.Div([
                    html.H1(
                        children='Table',
                        style={
                            'textAlign': 'center',
                            'color': 'green'
                        }), dash_table.DataTable(
                        id='table',
                        data=df_sum_state.to_dict("rows"),
                        columns=[{"name": i, "id": i}
                                 for i in df_sum_state.columns],
                        style_table=style_table,
                        style_cell=style_cell,
                        style_data_conditional=style_data_conditional,
                        style_header=style_header,
                        style_cell_conditional=style_cell_conditional,
                    )
                ], className="four columns"),
                html.Div([
                    html.H3('Fires per state in Brazil',
                            style={
                                'textAlign': 'center',
                                'color': 'green'}),
                    dcc.Graph(
                        id='Graph1',
                        figure={
                            'data': [
                                {'x': df.state, 'y': df.number, 'type': 'bar', 'name': 'state', 'marker': {'color': [
                                    '#b50000', '#e63535', '#fa8989'], 'size': 9, 'line': {'width': 1, 'color': 'orange'}}},
                            ]
                        }
                    )
                ], className="eight columns"),
            ], className="row")
        ])

    elif tab == 'tab-2':
        return html.Div([
            html.Div([
                html.Div([
                    html.H1(
                        children='Table',
                        style={
                            'textAlign': 'center',
                            'color': 'green'
                        }), dash_table.DataTable(
                        id='table',
                        data=df_sum_year.to_dict("rows"),
                        columns=[{"name": i, "id": i}
                                 for i in df_sum_year.columns],
                        style_table=style_table,
                        style_cell=style_cell,
                        style_data_conditional=style_data_conditional,
                        style_header=style_header,
                        style_cell_conditional=style_cell_conditional,
                    )

                ], className="four columns"),

                html.Div([
                    html.H3('Fires in Brazil per year',
                            style={
                                'textAlign': 'center',
                                'color': 'green'}
                            ), dcc.Graph(
                        id='Graph1',
                        figure={
                            'data': [
                                {'x': df.year, 'y': df.number, 'type': 'bar', 'name': 'state', 'marker': {'color': [
                                    '#b50000', '#e63535', '#fa8989'], 'size': 9, 'line': {'width': 1, 'color': 'orange'}}},

                            ]

                        }

                    )
                ], className="eight columns"),
            ], className="row")
        ])

    else:

        return html.Div([
            html.Div([
                html.Div([
                    html.H1(
                        children='Table',
                        style={
                            'textAlign': 'center',
                            'color': 'green'
                        }), dash_table.DataTable(
                            id='table',
                            data=df_sum_yr_month.to_dict("rows"),
                            columns=[{"name": i, "id": i}
                                     for i in df_sum_yr_month.columns],
                            style_table=style_table,
                            style_cell=style_cell,
                            style_data_conditional=style_data_conditional,
                            style_header=style_header,
                            style_cell_conditional=style_cell_conditional,
                    )

                ], className="four columns"),

                html.Div([
                    html.H3('Forest fires per month and year', style={
                        'textAlign': 'center',
                        'color': 'green'}
                    ), dcc.Graph(
                        id='Graph1',
                        figure={
                            'data': [
                                {'x': df.key, 'y': df.number, 'type': 'bar', 'name': 'state', 'marker': {'color': [
                                    '#b50000', '#e63535', '#fa8989'], 'size': 9, 'line': {'width': 1, 'color': 'orange'}}},

                            ],

                        }
                    )
                ], className="eight columns"),
            ], className="row")
        ])


if __name__ == '__main__':
    app.run_server(debug=True)


Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Debugger PIN: 551-621-761
Debugger PIN: 551-621-761
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


### Overview of the app

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "img/state.png")

In [ ]:
Image(url= "img/year.png")

In [ ]:
Image(url= "img/month.png")